# Regression in Python

This is a very quick run-through of some statistical concepts

* Regression Models
    * Linear, Logistic (also Diagnostics)
* Boston Housing data set
    * Predict housing prices
* Some resampling methods    
    * Train-Test splits
    * Cross Validation
* Prediction, Explanation, Desription

* Linear regression
    * Multiple ways to do this in python: `numpy`, `scipy`, `statsmodels`, `sklearn`


The packages we'll cover are: `statsmodels`, `seaborn`, and `scikit-learn`.


In [3]:
# special IPython command to prepare the notebook for matplotlib
%matplotlib inline 

import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import sklearn

# special matplotlib argument for improved plots
from matplotlib import rcParams



## Linear Regression
### A Brief recap

[Linear Regression](http://en.wikipedia.org/wiki/Linear_regression) is a method to model the relationship between a set of independent variables $X$ (also knowns as explantory variables, features, predictors) and a dependent variable $Y$.  This method assumes the relationship bewteen each predictor $X$ is linearly related to the dependent variable $Y$.  

$$ Y = \beta_0 + \beta_1 X + \epsilon$$

where $\epsilon$ is considered as an unobservable random variable that adds noise to the linear relatiosnhip. This is the simplest form of linear regression (one variable), we'll call this the simple model. 

* $\beta_0$ is the intercept of the linear model

* Multiple linear regression is when you have more than one independent variable
    * $X_1$, $X_2$, $X_3$, $\ldots$

$$ Y = \beta_0 + \beta_1 X_1 + \ldots + \beta_p X_p + \epsilon$$ 

* Back to the simple model. The model in linear regression is the *conditional mean* of $Y$ given the values in $X$ is expressed a linear function.  

$$ y = f(x) = E(Y | X = x)$$ 

![conditional mean](images/conditionalmean.png)
http://www.learner.org/courses/againstallodds/about/glossary.html

* The goal is to estimate the coefficients (e.g. $\beta_0$ and $\beta_1$). We represent the estimates of the coefficients with a "hat" on top of the letter.  

$$ \hat{\beta}_0, \hat{\beta}_1 $$

* Once you estimate the coefficients $\hat{\beta}_0$ and $\hat{\beta}_1$, you can use these to predict new values of $Y$

$$\hat{y} = \hat{\beta}_0 + \hat{\beta}_1 x_1$$


* How do you estimate the coefficients? 
    * There are many ways to fit a linear regression model
    * The method called **least squares** is one of the most common methods
    * We will discuss least squares today
    
#### Least squares
[Least squares](http://en.wikipedia.org/wiki/Least_squares) is a method that can estimate the cofficients of a linear model by minimizing the difference between the following: 

$$ S = \sum_{i=1}^N r_i = \sum_{i=1}^N (y_i - (\beta_0 + \beta_1 x_i))^2 $$

where $N$ is the number of observations.  

* We will not go into the mathematical details, but the least squares estimates $\hat{\beta}_0$ and $\hat{\beta}_1$ minimize the sum of the squared residuals $r_i = y_i - (\beta_0 + \beta_1 x_i)$ in the model (i.e. makes the difference bewteen the observed $y_i$ and linear model $\beta_0 + \beta_1 x_i$ as small as possible). 

#### Purposes of linear regression

Given a dataset $X$ and $Y$, linear regression can be used to: 

* Build a predictive model to predict future values of $X_i$ without a $Y$ value.  
* Model the strength of the relationship between each dependent variable $X_i$ and $Y$
    * Sometimes not all $X_i$ will have a relationship with $Y$
    * Need to figure out which $X_i$ contributes most information to determine $Y$ 




For this lab, we're going to consider a few important statistical models that are important in many fields. The first of these is linear regression, a metho

Linear regression is used in so many applications that I won't warrant this with examples. It is in many cases, the first pass prediction algorithm for continuous outcomes. 

# Boston Housing Data Set

The [Boston Housing data set](https://archive.ics.uci.edu/ml/datasets/Housing) contains information about the housing values in suburbs of Boston.  This dataset was originally taken from the StatLib library which is maintained at Carnegie Mellon University and is now available on the UCI Machine Learning Repository. 

## Load the Boston Housing data set from `sklearn`

This data set is available in the [sklearn](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html#sklearn.datasets.load_boston) python module which is how we will access it today.  

In [ ]:
from sklearn.datasets import load_boston
boston = load_boston()

In [ ]:
boston.keys()

In [ ]:
boston.data.shape

In [ ]:
# Print column names
print boston.feature_names

In [ ]:
# Print description of Boston housing data set
print boston.DESCR

Now let's explore the data set itself. 

In [ ]:
bos = pd.DataFrame(boston.data)
bos.head()

There are no column names in the DataFrame. Let's add those. 

In [ ]:
bos.columns = boston.feature_names[:-1]
bos.head()

Now we have a pandas DataFrame called `bos` containing all the data we want to use to predict Boston Housing prices.  Let's create a variable called `PRICE` which will contain the prices. This information is contained in the `target` data. 

In [ ]:
print boston.target.shape

In [ ]:
bos['PRICE'] = boston.target

# EDA and Summary Statistics

Let's explore this data set.  First we use `describe()` to get basic summary statistics for each of the columns. 

In [ ]:
bos.describe()

#### Scatter plots
Let's look at some scatter plots for three variables: 'CRIM', 'RM' and 'PTRATIO'. 

What kind of relationship do you see? e.g. positive, negative?  linear? non-linear? 

In [ ]:
plt.scatter(bos.CRIM, bos.PRICE)
plt.xlabel("Per capita crime rate by town (CRIM)")
plt.ylabel("Housing Price")
plt.title("Relationship between CRIM and Price")

In [ ]:
plt.scatter(bos.RM, bos.PRICE)
plt.xlabel("Average number of rooms per dwelling (RM)")
plt.ylabel("Housing Price")
plt.title("Relationship between RM and Price")

In [ ]:
plt.scatter(bos.PTRATIO, bos.PRICE)
plt.xlabel("Pupil-to-Teacher Ratio (PTRATIO)")
plt.ylabel("Housing Price")
plt.title("Relationship between PTRATIO and Price")

#### Histograms

In [ ]:
plt.hist(bos.CRIM)
plt.title("CRIM")
plt.xlabel("Crime rate per capita")
plt.ylabel("Frequencey")
plt.show()

In [ ]:
plt.hist(bos.PRICE)
plt.title('Housing Prices: $Y_i$')
plt.xlabel('Price')
plt.ylabel('Frequency')
plt.show()

If we are interested in predicting price, we need to build some kind of model. Let's use linear regression to predict price using the 13 variables in the `bos` DataFrame.  

There are several ways of doing this in python. One way of doing this is to use `sklearn`.  

# `statsmodels`

[statsmodels](http://statsmodels.sourceforge.net) is python module specifically for estimating statistical models (less machine learning compared to `sklearn`). It can estimate many types of statistical models, but today we will focus on linear regression. 

### Recap of linear regression and least squares
Last week we learned, [linear regression](http://en.wikipedia.org/wiki/Linear_regression) is a method to model the relationship between a set of independent variables $X$ (also knowns as explantory variables, features, predictors) and a dependent variable $Y$.  This method assumes the relationship bewteen each predictor $X$ is linearly related to the dependent variable $Y$.  

$$ Y = \beta_0 + \beta_1 X + \epsilon$$

On Tuesday in lecture, we learned how to write this model using matrix multiplication 

$$ Y = \beta X + \epsilon$$ 

where $Y$ has dimensions $n \times 1$, $X$ has dimensions $n \times p$ and $\epsilon$ has dimensions $n \times 1$.  On Tuesday, we also derived the [least squares](http://en.wikipedia.org/wiki/Least_squares) estimates of the coefficients of a linear model. These estimates minimize the difference between the following: 

$$ S = \sum_{i=1}^n r_i = \sum_{i=1}^n (y_i - (\beta_0 + \beta_1 x_i))^2 $$

where $n$ is the number of observations.  

> The least squares estimates $\hat{\beta}_0$ and $\hat{\beta}_1$ minimize the sum of the squared residuals $r_i = y_i - (\beta_0 + \beta_1 x_i)$ in the model (i.e. makes the difference bewteen the observed $y_i$ and linear model $\beta_0 + \beta_1 x_i$ as small as possible). 

### Old Faithful Geyser Data Set

The [Old Faithful Geyser](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/faithful.html) data set is a well-known data set that depicts the relationship of the waiting time between eruptions and the duration of the eruption for the Old Faithful geyser in Yellowstone National Park, Wyoming, USA [[webcam]](http://yellowstone.net/webcams/). This data set is found in the base installation of the [R programming language](http://cran.r-project.org).  

`faithful` is a data set with 272 observations on 2 variables.

Column name| Description 
--- | --- 
eruptions | Eruption time (in mins)
waiting	| Waiting time to next eruption (in mins)

There is a function in `statsmodels` (or `sm` for short) called `sm.datasets.get_rdataset` which will download and return a data set found in [R](http://cran.r-project.org).  

Let's import the `faithful` dataset. 

In [ ]:
faithful = sm.datasets.get_rdataset("faithful")

In [ ]:
# Let's look at the help file
# sm.datasets.get_rdataset?
# faithful?

In [ ]:
faithful.title

In [ ]:
faithful = faithful.data
faithful.head()

In [ ]:
faithful.shape

# Histogram 

Create a histogram of the time between eruptions. What do you see? 

In [ ]:
plt.hist(faithful.waiting)
plt.xlabel('Waiting time to next eruption (in mins)')
plt.ylabel('Frequency')
plt.title('Old Faithful Geyser time between eruption')
plt.show()

This histogram indicates [Old Faithful isn’t as “faithful” as you might think](http://people.stern.nyu.edu/jsimonof/classes/2301/pdf/geystime.pdf). 

### Scatter plot 

Create a scatter plot of the `waiting` on the x-axis and the `eruptions` on the y-axis. 

In [ ]:
plt.scatter(faithful.waiting, faithful.eruptions)
plt.xlabel('Waiting time to next eruption (in mins)')
plt.ylabel('Eruption time (in mins)')
plt.title('Old Faithful Geyser')
plt.show()


### Build a linear regression to predict eruption time using `statsmodels`

Now let's build a linear regression model for the `faithful` DataFrame, and estimate the next eruption duration if the waiting time since the last eruption has been 75 minutes.

$$ Eruptions = \beta_0 + \beta_1 * Waiting + \epsilon $$ 

In [ ]:
X = faithful.waiting
y = faithful.eruptions
model = sm.OLS(y, X)

In [ ]:
# Let's look at the options in model
# model.<tab>

In [ ]:
results = model.fit()

In [ ]:
# Let's look at the options in results
# results.<tab>

In [ ]:
print results.summary()

In [ ]:
results.params.values

We notice, there is no intercept ($\beta_0$) fit in this linear model.  To add it, we can use the function `sm.add_constant`.  

In [ ]:
X = sm.add_constant(X)
X.head()

Now let's fit a linear regression model with an intercept. 

In [ ]:
modelW0 = sm.OLS(y, X)
resultsW0 = modelW0.fit()
print resultsW0.summary()

If you want to predict the time to the next eruption using a waiting time of 75, you can directly estimate this using the equation 

$$ \hat{y} = \hat{\beta}_0 + \hat{\beta}_1 * 75 $$ 

or you can use `results.predict`.  

In [ ]:
newX = np.array([1,75])
resultsW0.params[0]*newX[0] + resultsW0.params[1] * newX[1]

In [ ]:
resultsW0.predict(newX)

Based on this linear regression, if the waiting time since the last eruption has been 75 minutes, we expect the next one to last approximately 3.80 minutes.

### Plot the regression line 

Instead of using `resultsW0.predict(X)`, we can use `resultsW0.fittedvalues` which are the $\hat{y}$. 

In [ ]:
plt.scatter(faithful.waiting, faithful.eruptions)
plt.xlabel('Waiting time to next eruption (in mins)')
plt.ylabel('Eruption time (in mins)')
plt.title('Old Faithful Geyser')

plt.plot(faithful.waiting, resultsW0.fittedvalues, color='blue', linewidth=3)
plt.show()


### Residuals, residual sum of squares, mean squared error

Recall, we can directly calculate the residuals as 

$$r_i = y_i - (\hat{\beta}_0 + \hat{\beta}_1 x_i)$$

To calculate the residual sum of squares, 

$$ S = \sum_{i=1}^n r_i = \sum_{i=1}^n (y_i - (\hat{\beta}_0 + \hat{\beta}_1 x_i))^2 $$

where $n$ is the number of observations.  Alternatively, we can simply ask for the residuals using `resultsW0.predict`

In [ ]:
resids = faithful.eruptions - resultsW0.predict(X)


In [ ]:
resids = resultsW0.resid


In [ ]:
plt.plot(faithful.waiting, resids, 'o')
plt.hlines(y = 0, xmin=40, xmax = 100)
plt.xlabel('Waiting time')
plt.ylabel('Residuals')
plt.title('Residual Plot')
plt.show()

The residual sum of squares: 

In [ ]:
print np.sum((faithful.eruptions - resultsW0.predict(X)) ** 2)

Mean squared error: 

In [ ]:
print np.mean((faithful.eruptions - resultsW0.predict(X)) ** 2)

### Build a linear regression to predict eruption time using least squares 

Now let's build a linear regression model for the `faithful` DataFrame, but instead of using `statmodels` (or `sklearn`), let's use the least squares estimates of the coefficients for the linear regression model.

$$ \hat{\beta} = (X^{\top}X)^{-1} X^{\top}Y $$ 

The `numpy` function [`np.dot`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html#numpy.dot) is the dot product (or inner product) of two vectors (or arrays in python).  

The `numpy` function [`np.linalg.inv`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.inv.html#numpy.linalg.inv) can be used to compute the inverse of a matrix. 

In [ ]:
X = sm.add_constant(faithful.waiting)
y = faithful.eruptions


First, compute $X^{\top}X$


In [ ]:
np.dot(X.T, X)


Next, compute the inverse of $X^{\top}X$ or $(X^{\top}X)^{-1}$. 

In [ ]:
np.linalg.inv(np.dot(X.T, X))

Finally, compute $\hat{\beta} = (X^{\top}X)^{-1} X^{\top}Y $

In [ ]:
beta = np.linalg.inv(np.dot(X.T, X)).dot(X.T).dot(y)
print "Directly estimating beta:", beta
print "Estimating beta using statmodels: ", resultsW0.params.values

# Let's try an example together

We will revisit the Motor Trend Car Road Tests data set used in [Lab 3](http://nbviewer.ipython.org/github/cs109/2014/blob/master/labs/Lab3_Notes.ipynb) and build a linear regression model to predict miles per gallon (`mpg`). 

## Motor Trend Car Road Tests Data

We previously looked at the [mtcars](https://stat.ethz.ch/R-manual/R-devel/library/datasets/html/mtcars.html) data set in [Lab 3](http://nbviewer.ipython.org/github/cs109/2014/blob/master/labs/Lab3_Notes.ipynb). The data was extracted from the 1974 Motor Trend US magazine, and comprises fuel consumption and 10 aspects of automobile design and performance for 32 automobiles (1973–74 models). This data set is also found in the base installation of the [R programming language](http://cran.r-project.org).  

Column name | Description 
--- | --- 
mpg | Miles/(US) gallon
cyl | Number of cylinders
disp | Displacement (cu.in.)
hp | Gross horsepower
drat | Rear axle ratio
wt | Weight (lb/1000)
qsec | 1/4 mile time
vs | V/S
am | Transmission (0 = automatic, 1 = manual)
gear | Number of forward gears
carb | Number of carburetors

First, read in the `mtcars` data set using the `sm.datasets.get_rdataset` function to import the dataset from R. 

In [ ]:
# your turn
mtcars = sm.datasets.get_rdataset("mtcars")
mtcars = mtcars.data

### Histogram

In [ ]:
mtcars['mpg'].hist()
plt.title('Distribution of MPG')
plt.xlabel('Miles Per Gallon')

### Scatter plots

Relationship between `cyl` and `mpg`

In [ ]:
plt.plot(mtcars.cyl, mtcars.mpg, 'o')
plt.xlim(3, 9)
plt.xlabel('Cylinders')
plt.ylabel('MPG')
plt.title('Relationship between cylinders and MPG')

Relationship between `horsepower` and `mpg`

In [ ]:
plt.plot(mtcars.hp, mtcars.mpg, 'o')
plt.xlabel('Horsepower')
plt.ylabel('MPG')
plt.title('Relationship between horsepower and MPG')

### Build a linear regression to predict mpg using `statsmodels`

Now let's build a linear regression model for the `mtcars` DataFrame, and estimate predicted `mpg` given a new car has 6 cylinders and 180 horsepower.  

$$ MPG = \beta_0 + \beta_1 * cylinders + \beta_2 * horsepower + \epsilon $$ 

In [ ]:
# your turn
y = mtcars.mpg
X = mtcars[['cyl', 'hp']]
X = sm.add_constant(X)

model = sm.OLS(y,X)
results = model.fit()
print results.summary()

In [ ]:
newX = np.array([1, 6, 180])
results.predict(newX)

What if a new car had 4 cylinders and 120 horsepower? 

In [ ]:
# your turn
newX = np.array([1, 4, 120])
results.predict(newX)

Now estimate the least squares estimates for $\beta_0$, $\beta_1$ and $\beta_2$ using matrix multiplication and the formula: 

$$ \hat{\beta} = (X^{\top}X)^{-1} X^{\top}Y $$

In [ ]:
# your turn
beta = np.linalg.inv(np.dot(X.T, X)).dot(X.T).dot(y)
beta

# Many different types of regression

You do not always have a continuous $y$ variable that you are measuring.  Sometimes it may be binary (e.g. 0 or 1). Sometimes it may be count data.  What do you do?

Use other types of regression besides just simple linear regression.  

[Nice summary of several types of regression](http://www.datasciencecentral.com/profiles/blogs/10-types-of-regressions-which-one-to-use). 

## Logistic Regression

## Regression Diagnostics (linear)

## Prediction, Explanation, Desription

## Cross Validation

***

<div class="span7 alert alert-info">
asdasdasd
</div>

<div class="span5 alert alert-danger">
asaas
</div>
